In [0]:
%sql

-- Base Clima: fire_risk_project.02_silver.silver_open_meteo
-- Base Incendios: fire_risk_project.02_silver.silver_nasa_firms
-- Destino Gold: fire_risk_project.03_gold.gold_dataset_full

CREATE OR REPLACE TABLE fire_risk_project.03_gold.gold_dataset_full AS

WITH fires_snapped AS (
    SELECT 
        fecha_join, 
        hora_join,
        ROUND(latitude, 1) AS lat_centroid,
        ROUND(longitude, 1) AS long_centroid,
        
        -- Features del incendio
        MAX(confidence) as confidence,
        MAX(frp) as frp,
        1 AS target
    FROM fire_risk_project.02_silver.silver_nasa_firms
    GROUP BY 
      fecha_join,
      hora_join,
      lat_centroid,
      long_centroid
),

climate_grid AS (
    SELECT 
        grid_id,
        timestamp_clima,
        fecha_join,
        hora_join,
        
        -- REDONDEO POR LAS DUDAS
        ROUND(latitude, 1) AS latitude,
        ROUND(longitude, 1) AS longitude,
        elevation,
        temperature_2m,
        relative_humidity_2m,
        vapour_pressure_deficit,
        precipitation,
        wind_speed_10m,
        wind_direction_10m,
        wind_gusts_10m,
        et0_fao_evapotranspiration,
        soil_moisture_0_to_7cm,
        soil_moisture_28_to_100cm,
        weather_code
    FROM fire_risk_project.02_silver.silver_open_meteo 
)

-- 3. EL CRUCE FINAL
SELECT 
    c.grid_id, 
    c.timestamp_clima,
    c.latitude, 
    c.longitude, 
    c.elevation,
    -- Features Climáticos
    c.temperature_2m,
    c.relative_humidity_2m,
    c.vapour_pressure_deficit,
    c.precipitation,
    c.wind_speed_10m,
    c.wind_direction_10m,
    c.wind_gusts_10m,
    c.et0_fao_evapotranspiration,
    c.soil_moisture_0_to_7cm,
    c.soil_moisture_28_to_100cm,
    c.weather_code,
    
    -- Features del Incendio (Cruzados)
    COALESCE(f.target, 0) as target,       
    COALESCE(f.frp, 0) as fire_intensity,  
    CASE 
        WHEN f.confidence = 'l' THEN 1
        WHEN f.confidence = 'n' THEN 2
        WHEN f.confidence = 'h' THEN 3
        ELSE 0 -- Para que no sea null, queda en 0
    END as fire_confidence

FROM climate_grid c
LEFT JOIN fires_snapped f
    ON c.fecha_join = f.fecha_join
    AND c.hora_join = f.hora_join
    AND ROUND(c.latitude, 1) = f.lat_centroid
    AND ROUND(c.longitude, 1) = f.long_centroid;

In [0]:
%sql

SELECT * FROM fire_risk_project.03_gold.gold_dataset_full LIMIT 50;

In [0]:
%sql
-- Validaciones

In [0]:
%sql
SELECT 
    COUNT(target)
FROM fire_risk_project.03_gold.gold_dataset_full
WHERE target = 1;

In [0]:
%sql
SELECT count(*) as filas_con_nulos
FROM fire_risk_project.03_gold.gold_dataset_full
WHERE fire_confidence IS NULL;

In [0]:
%sql
SELECT * FROM fire_risk_project.03_gold.gold_dataset_full
WHERE 
    timestamp_clima = '2020-08-23 18:00:00'
    AND latitude = -31.2 
    AND longitude = -64.4;